In [106]:
import pandas as pd
import numpy as np
import os

In [ ]:
# Read characters from the Comic Vine Database
df_characters = pd.read_json("../ComicVine/characters.json")
df_timeline = pd.DataFrame()

In [ ]:
# Select the minority characters' name and add them to a new dataframe
minority_characters = ['Black Panther', 'Luke Cage', 'Misty Knight', 'Shang-Chi', 'Colleen Wing', 'Northstar', 'Dust (Qadir)', 'Mikaal Tomas', 'Black Racer', 'John Stewart', 'Ice Maiden', 'Silhouette']

for character in minority_characters:
    if character == "Silhouette":
        df_timeline = df_timeline.append(df_characters.loc[df_characters['real_name'] == "Ursula Zandt"].iloc[0])
    else:
        df_timeline = df_timeline.append(df_characters.loc[df_characters['name'] == character].iloc[0])

df_timeline = df_timeline[['aliases', 'api_detail_url', 'deck', 'first_appeared_in_issue', 'gender', 'image', 'name', 'publisher', 'real_name', 'site_detail_url']]

df_timeline['gender'] = df_timeline['gender'].map({1: 'Male', 2: 'Female', 3: 'Other'})
df_timeline['image'] = df_timeline.apply(lambda row: row.image['super_url'], axis=1)
df_timeline['publisher'] = df_timeline.apply(lambda row: row.publisher['name'], axis=1)

In [107]:
# save characters and read them
df_timeline.to_csv(path_or_buf='data/characters_timeline.csv')
df_characters = pd.read_csv("data/characters_timeline.csv")

In [109]:
# Read DC Comics characters from the fivethirtyeight Database
df1 = pd.read_csv("https://raw.githubusercontent.com/fivethirtyeight/data/master/comic-characters/dc-wikia-data.csv")
df1['publisher'] = "DC Comics"
df1.rename(str.lower, axis='columns', inplace=True)
df1.rename(columns={'id':'identity_type', 'name':'character_name'}, inplace=True)

In [110]:
# Read Marvel Comics characters from the fivethirtyeight Database
df2 = pd.read_csv("https://raw.githubusercontent.com/fivethirtyeight/data/master/comic-characters/marvel-wikia-data.csv")
df2['publisher'] = "MARVEL Comics"
df2.rename(str.lower, axis='columns', inplace=True)
df2.rename(columns={'id':'identity_type', 'name':'character_name'}, inplace=True)

In [ ]:
# concat the two fivethirtyeight databases (Marvel + DC Comics)
df_fte = pd.concat([df1, df2], ignore_index=True)

In [112]:
def get_year_month(first_appearance):  # 1939, May
    datetime = pd.to_datetime(first_appearance, errors='ignore', yearfirst=True) 
    try:
        dt_str = datetime.strftime('%b-%y')  # e.g. 1939-05
        return dt_str
    except ValueError:
        return np.nan
    except AttributeError:
        if datetime == "1988, Holiday":
            datetime = "1988, December"  # I guess...
            datetime = pd.to_datetime(datetime, errors='ignore', yearfirst=True)  
            dt_str = datetime.strftime('%b-%y')  # e.g. 1939-05
            return dt_str
        if type(datetime) == str:
            datetime = pd.to_datetime(datetime, errors='ignore', yearfirst=True) 
            return datetime
        return datetime

In [114]:
# fix the first appearance date
df_fte['first appearance'] = df_fte.apply(lambda row: get_year_month(row['first appearance']), axis=1)
df_fte["first appearance"] = df_fte["first appearance"].str.split('-').str[0]
df_fte["first appearance year"] = df_fte["year"].fillna(0.0).astype(int).map(str) + "-"+ df_fte["first appearance"]

In [116]:
# select the names of the characters from the second database - they need to match the order of the first
minority_characters_458 = ["T'Challa (Earth-616)", 'Luke Cage (Earth-616)', "Mercedes Knight (Earth-616)", "Shang-Chi (Earth-616)", "Colleen Wing (Earth-616)", "Jean-Paul Beaubier (Earth-616)", "Sooraya Qadir (Earth-616)", "Mikaal Tomas (New Earth)", "William Walker (New Earth)", "John Stewart (New Earth)", "Sigrid Nansen (New Earth)", "Jake Ketchum (New Earth)"]

df_timeline_dates = pd.DataFrame()
for character in minority_characters_458:
    df_timeline_dates = df_timeline_dates.append(df_fte[df_fte["character_name"] == character])
    


In [117]:
df_timeline_dates[['year', 'first appearance year']]

,year,first appearance year
22263,1966.0,1966-Jul
6933,1972.0,1972-Jun
7050,1972.0,1972-Mar
7026,1973.0,1973-Dec
7128,1974.0,1974-Nov
7021,1979.0,1979-Apr
7232,2002.0,2002-Dec
339,1976.0,1976-Mar
684,1971.0,1971-Jul
29,1971.0,1971-Dec


In [118]:
df_characters.index = df_timeline_dates.index
df_characters[['year', 'first_appearance_date']] = df_timeline_dates[['year', 'first appearance year']]

In [119]:
# fix Silhouette
df_characters.loc[df_characters['name'] == "Silhouette","year"] = 1986
df_characters.loc[df_characters['name'] == "Silhouette","first_appearance_date"] = "1986-Sep"

In [120]:
df_characters.to_csv(path_or_buf='data/characters_timeline.csv')
df_characters.to_json(path_or_buf='data/characters_timeline.json', orient='records')

In [121]:
# add decade column to each character based on the year they appeared
df_characters['decades'] = "The " + df_characters['year'].astype(str).str[2] + "0s"

In [122]:
df_characters

,Unnamed: 0,aliases,api_detail_url,deck,description,first_appeared_in_issue,gender,image,name,publisher,real_name,site_detail_url,year,first_appearance_date,decades
22263,179,King T'Challa\r\r\nThe Most Dangerous Man Aliv...,https://comicvine.gamespot.com/api/character/4...,"T'Challa is the Black Panther, king of Wakanda...",<h2><strong>Origin</strong></h2><figure data-a...,{'api_detail_url': 'https://comicvine.gamespot...,Male,https://comicvine.gamespot.com/api/image/scale...,Black Panther,Marvel,T'Challa,https://comicvine.gamespot.com/black-panther/4...,1966.0,1966-Jul,The 60s
6933,155,Carl Lucas\r\nHero for Hire\r\nPower Man,https://comicvine.gamespot.com/api/character/4...,"Luke Cage, wrongly convicted and unjustly impr...","<h2>Origin</h2><figure data-align=""right"" data...",{'api_detail_url': 'https://comicvine.gamespot...,Male,https://comicvine.gamespot.com/api/image/scale...,Luke Cage,Marvel,Carl Lucas,https://comicvine.gamespot.com/luke-cage/4005-...,1972.0,1972-Jun,The 70s
7050,190,Maya Corday\r\r\nControl,https://comicvine.gamespot.com/api/character/4...,"A former NYPD cop with a bionic arm, martial a...",<h2>Current Events</h2><p>Misty Knight is pres...,{'api_detail_url': 'https://comicvine.gamespot...,Female,https://comicvine.gamespot.com/api/image/scale...,Misty Knight,Marvel,Mercedes Knight,https://comicvine.gamespot.com/misty-knight/40...,1972.0,1972-Mar,The 70s
7026,8230,Master of Kung Fu\r\r\nSon of Fu Manchu\r\r\nT...,https://comicvine.gamespot.com/api/character/4...,Shang-Chi is the son of a sinister Chinese mas...,"<h2>Origin</h2><figure data-align=""right"" data...",{'api_detail_url': 'https://comicvine.gamespot...,Male,https://comicvine.gamespot.com/api/image/scale...,Shang-Chi,Marvel,Zheng Shang-Chi,https://comicvine.gamespot.com/shang-chi/4005-...,1973.0,1973-Dec,The 70s
7128,193,Daughter of the Dragon,https://comicvine.gamespot.com/api/character/4...,Colleen Wing is a samurai and one half of the ...,"<h2>Origin</h2><figure data-align=""right"" data...",{'api_detail_url': 'https://comicvine.gamespot...,Female,https://comicvine.gamespot.com/api/image/scale...,Colleen Wing,Marvel,Colleen Wing,https://comicvine.gamespot.com/colleen-wing/40...,1974.0,1974-Nov,The 70s
7021,1290,Jean-Paul Beaubier \r\r\nJean-Paul Martin,https://comicvine.gamespot.com/api/character/4...,"Twin brother of Aurora, Northstar is the haugh...","<h2>Origin</h2><figure data-align=""right"" data...",{'api_detail_url': 'https://comicvine.gamespot...,Male,https://comicvine.gamespot.com/api/image/scale...,Northstar,Marvel,Jean-Paul Beaubier,https://comicvine.gamespot.com/northstar/4005-...,1979.0,1979-Apr,The 70s
7232,2271,Sooraya Qadir\r\r\nTuraab,https://comicvine.gamespot.com/api/character/4...,Dust was a student at the Xavier Institute and...,<h2>Origin</h2><p>Dust is a mutant from <a hre...,{'api_detail_url': 'https://comicvine.gamespot...,Female,https://comicvine.gamespot.com/api/image/scale...,Dust (Qadir),Marvel,Sooraya Qadir,https://comicvine.gamespot.com/dust-qadir/4005...,2002.0,2002-Dec,The 00s
339,16113,Blue Man\r\nCosmic Geek\r\nGreg Bailey\r\nMich...,https://comicvine.gamespot.com/api/character/4...,The last of his alien warrior race and the thi...,<h2>Origin</h2><p>Mikaal Tomas comes from a ra...,{'api_detail_url': 'https://comicvine.gamespot...,Male,https://comicvine.gamespot.com/api/image/scale...,Mikaal Tomas,DC Comics,Mikaal Tomas,https://comicvine.gamespot.com/mikaal-tomas/40...,1976.0,1976-Mar,The 70s
684,6267,Death\r\nGrim Reaper\r\nSergeant Willie Walker...,https://comicvine.gamespot.com/api/character/4...,The Grim Reaper of the DC Universe. When it i...,"<figure data-align=""right"" data-img-src=""https...",{'api_detail_url': 'https://comicvine.gamespot...,Male,https://comicvine.gamespot.com/api/image/scale...,Black Racer,DC Comics,Willie Walker,https://comicvine.gamespot.com/black-racer/400...,1971.0,1971-Jul,The 70s
29,6549,John Stewart\r\nGreen Lantern\r\nGreen Lantern...,https://comicvine.gamespot.com/api/ch

In [123]:
# write the historic background of each decade
the60sText = "At the dawn of the Marvel age of comics, the Civil Rights movement was in full swing in America.<br/> Initially, African-American characters were rarely seen in Marvel publications, but they slowly started making their appearances as background characters";
the70sText = "During the 1970s in the United States, cultural trends included the popularization of Blaxploitation films and Kung-Fu epics. These films featured African Americans and Asian characters as protagonists, though their portrayals varied from being heroes and antiheroes to villains.<br/><br/>The 1970's were a time of great social activism and protest against the government, as the still-ongoing Vietnam War (it started in 1955 and ended in 1975) was increasingly unpopular with the citizens and political scandals such as Watergate led the public to disillusionment with the country's leadership and direction. The Marvel Comics staff were influenced by these movements, and their stories often reflected their ideas. In their stories, they often chose to champion the underdog and the marginalized.\n ";
the80sText = "Even more ethnic characters into the Marvel Universe, without any overt racial stereotypes being perpetuated.\n\nStereotypical depictions became more rare and more muted, although there were still some exceptions.<br/><br/>One notable discrimination of sorts was the depiction of homosexuality in comics up to this era, or rather the lack of mention.";
the90sText = "Most racial story-lines in the early 1990s were centered around drug and gang culture, which was a recurring problem in the United States during the time, notably in the Los Angeles area.";
the00sText = "The issue or race became a term of polarization in many of the texts, following the terrorist attacks on the World Trade Center on September 11th, 2001.<br/><br/>Homosexuality was fully out of the closet and embraced, with many diverse gay and lesbian characters introduced, adding more dimension for bigoted antagonists who faced Marvel's various heroes and supporting cast.";

In [124]:
# apply the text to the correct dataframe 
conditions = [
    (df_characters['decades'] == 'The 60s'),
    (df_characters['decades'] == 'The 70s'),
    (df_characters['decades'] == 'The 80s'),
    (df_characters['decades'] == 'The 90s'),
    (df_characters['decades'] == 'The 00s')]
choices = [the60sText, the70sText, the80sText, the90sText, the00sText]
df_characters['decade_background'] = np.select(conditions, choices, default='black')

In [125]:
df_characters.to_json(path_or_buf='data/characters_timeline.json', orient='records')

In [128]:
# write backround description of each character
df_characters.loc[df_characters['name'] == 'Black Panther', 'character_background'] = "Shortly after his introduction in 1966, the Black Panther was briefly renamed the Black Leopard to distance the character from the real-life revolutionary leftist group Black Panther Party, which was highly controversial at the time. However this name change was quickly reversed as Marvel decided that any connections between the comic-book character and the group were a stretch at best."
df_characters.loc[df_characters['name'] == 'Luke Cage', 'character_background'] = "Luke Cage was created shortly after blaxploitation films emerged as a popular new genre. The character was the first black superhero to star in his own comic-book series."
df_characters.loc[df_characters['name'] == 'Misty Knight', 'character_background'] = "The original depictions of Knight, a bionically enhanced black female detective with martial arts skills, were strongly influenced by the blaxploitation and Kung Fu crazes of the 1970s. Misty Knight and Iron Fist had the first interracial kiss between super heroes in mainstream comics in 1977." 
df_characters.loc[df_characters['name'] == 'Shang-Chi', 'character_background'] = "Shang-Chi was created in the 1970s, right around the time of fellow Marvel martial artist, Iron Fist. The main difference between the two lies mostly in the fact that while Danny Rand is a white American who just happens to learn martial arts, Shang-Chi is Chinese and has trained since childhood.  He is scheduled to appear in the first Marvel superhero movie featuring an Asian protagonist."
df_characters.loc[df_characters['name'] == 'Colleen Wing', 'character_background'] = "Having appeared in the Netflix series Iron Fist recently, Wing has been around since 1974, when she made her debut saving Iron Fist in a fight against a Japanese cult. A martial arts expert, Wing would go on to team up with Misty Knight, a woman of African-American descent, in an investigative team that took action when necessary. That would mark the first women-of-colour team-up in American comics." 
df_characters.loc[df_characters['name'] == 'Northstar', 'character_background'] = "Τhe first attempt at introducing an openly gay character came in from John Byrne's Alpha Flight run. Alpha Flight #7 (1984) was an attempt to reveal that the character Northstar was gay. However, this was downplayed and only subtle hints were given. The character was not fully \"outted\" until Alpha Flight #106 (1992)."
df_characters.loc[df_characters['name'] == 'Dust (Qadir)', 'character_background'] = "This era saw the introduction of new Muslim characters, such as the X-Man character Dust who wears a full burqa. In line with Sooraya's character as a traditional Muslim, she chooses to observe Islamic hijab by donning traditional dress. "
df_characters.loc[df_characters['name'] == 'Mikaal Tomas', 'character_background'] = "There have been several incarnations of Starman since his first appearance in 1941. Mikaal Tomas was 1970s version of Starman. It should come as no surprise that he wasn’t originally conceived as Gay. In fact he was only identified as Gay in 2010. This is after a twelve year (in comic book years) relationship with his partner Tony."
df_characters.loc[df_characters['name'] == 'Black Racer', 'character_background'] = "DC introduced its first minority superhero the Black Racer in June 1971. The corporeal form of the Black Racer is an African-American Vietnam veteran named Willie Walker. When Darkseid brought the war of the New Gods of New Genesis and Apokolips to Earth the Source, a sort of God like universal shared consciousness, contacted Walker and turned him into the deity known as the Black Racer."
df_characters.loc[df_characters['name'] == 'John Stewart', 'character_background'] = "John Stewart (Green Latern) was DC's next black superhero appeared in Green Lantern #87 published in December 1971."
df_characters.loc[df_characters['name'] == 'Ice Maiden', 'character_background'] = "Icemaiden is a rarity in the comic universe. She is a bisexual superhero. Bisexual identities are often ignored in media including comic books."
df_characters.loc[df_characters['name'] == 'Silhouette', 'character_background'] = "The Silhouette was one of the earliest Lesbian comic book characters. She is a Jewish character, a survivor of the Nazi regime and a lesbian. And she is thrown out of the Minutemen for being openly Gay (despite the fact that they kept members who were closeted). And eventually she was murdered in a hate crime."

In [129]:
# save file
df_characters.to_json(path_or_buf='data/characters_timeline.json', orient='records')